# Evaluate a pretrained detector on custom coco style dataset

In [1]:
from PIL import Image
import torch
import json
import numpy as np
import cv2
import torchvision.transforms.functional as F
import os
import sys
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

C:\Users\marce\anaconda3\envs\msc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


My imports

In [2]:
sys.path.append(os.path.join(sys.path[0], '../tooling/'))
from myloader import CocoDetection

### Prepare dataset

Visualizing and verifying dataset

In [4]:
img_dir = os.path.abspath("coco_train_m_1_False_gamedrill")
annotation = img_dir + "/annot.json"
dataset = CocoDetection(root=img_dir, annFile=annotation)
img, target = dataset[5]

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\marce\\Temp\\MSC\\msc_detector_evaluation\\notebook\\coco_train_m_1_False_gamedrill/annot.json'

In [ ]:
target

In [ ]:
x1 = target[0]["bbox"][0]
y1 = target[0]["bbox"][1]
x2 = x1+target[0]["bbox"][2]
y2 = y1+target[0]["bbox"][3]

display_img = cv2.rectangle(np.array(img), (x1, y1), (x2, y2), (255, 0, 0), 2)
F.to_pil_image(display_img)

### Prepare detector

Load pretrained model

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Perform inference for one test image

In [ ]:
result = model(img)

In [ ]:
result.show()

In [ ]:
result.pandas().xyxy[0] 

### Evaluate mAP

Perform inference on dataset and convert to coco style results

In [ ]:
call_list = np.arange(1,50).tolist()
im_id_list = []
preds = []

for call_no in call_list:
    print(call_no)

    img, target = dataset[call_no]
    im_id_list.append(target[0]['image_id'])
    
    result = model(img)
    detections = result.xyxy[0]  # Batch size of 1
    if len(detections)>0:
        for detect in detections:  # [x_min, y_min, x_max, y_max, score, class]
            scores = detect[4].tolist()
            
            boxes = detect[0:4].type(torch.int64).tolist()
            # from xyxy to xywh
            boxes[2] = boxes[2]-boxes[0]
            boxes[3] = boxes[3]-boxes[1]
            
            labels = detect[5].type(torch.int64).tolist()
            
            # Consider only ball detections
            if labels == 32:
                labels = 1  # Convert standard coco to custom coco
            else:
                continue
            
            preds.append(
                dict(
                    image_id = target[0]['image_id'],
                    category_id=labels,
                    bbox=boxes,
                    score=round(scores,3),
                )
            )

Save results

In [ ]:
resFile = "./detection_cocoresults.json"
with open(resFile, 'w') as out_file:
    json.dump(preds, out_file)

Load ground truth annotations and predictions

In [ ]:
cocoGt = COCO(annotation)

In [ ]:
cocoDt = cocoGt.loadRes(resFile)

Prepare evaluation function

In [ ]:
cocoEval = COCOeval(cocoGt=cocoGt, cocoDt=cocoDt, iouType='bbox')
cocoEval.params.imgIds  = im_id_list
cocoEval.params.useCats = True
cocoEval.params.catIds = 1

Compute mAP scores

In [ ]:
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()